In [412]:
import pandas as pd
import numpy as np
import hvplot.pandas
from sklearn.cluster import KMeans
import plotly.express as px

In [413]:
file_path_data = "Resources/Data/2021VAERSDATA.csv"
encoding = 'unicode_escape'
VAERS_DATA_df = pd.read_csv(file_path_data, encoding='unicode_escape')
VAERS_DATA_df=VAERS_DATA_df.set_index(["VAERS_ID"]).copy()
VAERS_DATA_df.head()

,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,DIED,DATEDIED,L_THREAT,ER_VISIT,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
VAERS_ID,,,,,,,,,,,,,,,,,,,,,
916710,1/1/2021,MO,23.0,23.0,NaN,F,NaN,NaN,Y,NaN,...,NaN,Hypothyroidism,NaN,NaN,2,1/1/2021,NaN,NaN,Y,NKDA
916720,1/1/2021,NY,23.0,23.0,NaN,M,NaN,NaN,NaN,NaN,...,NaN,NaN,unknown,NaN,2,1/1/2021,NaN,NaN,Y,NaN
916741,1/1/2021,AR,68.0,68.0,NaN,F,NaN,NaN,NaN,NaN,...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,NaN,NaN,2,1/1/2021,NaN,NaN,NaN,bee stings
916772,1/1/2021,GA,55.0,55.0,NaN,M,NaN,NaN,NaN,NaN,...,NaN,"HTN, Insomnia,High Cholesterol,",NaN,NaN,2,1/1/2021,NaN,Y,NaN,Codeine
916790,1/1/2021,TN,52.0,52.0,NaN,F,NaN,NaN,Y,NaN,...,NaN,"Asthma, migraines",NaN,NaN,2,1/1/2021,NaN,NaN,Y,"Sulfa, shellfish"


In [414]:
file_path_vax = "Resources/Data/2021VAERSVAX.csv"
encoding = 'unicode_escape'
VAERS_VAX_df = pd.read_csv(file_path_vax, encoding= 'unicode_escape')
VAERS_VAX_df=VAERS_VAX_df.set_index(["VAERS_ID"]).copy()
VAERS_VAX_df.head()

,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,
916710,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
916720,COVID19,MODERNA,011L20A,1,IM,AR,COVID19 (COVID19 (MODERNA))
916741,COVID19,PFIZER\BIONTECH,EH9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916772,COVID19,PFIZER\BIONTECH,EJ1685,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916790,COVID19,PFIZER\BIONTECH,NaN,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [415]:
VAERS_VAX_df=VAERS_VAX_df[VAERS_VAX_df.VAX_TYPE == "COVID19"].copy()
VAERS_VAX_df.head()

,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,
916710,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
916720,COVID19,MODERNA,011L20A,1,IM,AR,COVID19 (COVID19 (MODERNA))
916741,COVID19,PFIZER\BIONTECH,EH9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916772,COVID19,PFIZER\BIONTECH,EJ1685,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
916790,COVID19,PFIZER\BIONTECH,NaN,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [416]:
#Drop Unnecessary columns from VAERS_VAX_df
VAERS_VAX_df= VAERS_VAX_df.drop(["VAX_TYPE", "VAX_LOT", "VAX_ROUTE", "VAX_SITE", "VAX_NAME"], axis=1).copy()
VAERS_VAX_df.head()

,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,
916710,MODERNA,1
916720,MODERNA,1
916741,PFIZER\BIONTECH,1
916772,PFIZER\BIONTECH,UNK
916790,PFIZER\BIONTECH,1


In [417]:
file_path_symptoms = "Resources/Data/2021VAERSSYMPTOMS.csv"
encoding = 'unicode_escape'
VAERS_SYMPTOMS_df = pd.read_csv(file_path_symptoms, encoding= 'unicode_escape')
VAERS_SYMPTOMS_df.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916710,Appendicitis,23.1,Band neutrophil percentage increased,23.1,Surgery,23.1,White blood cell count increased,23.1,NaN,NaN
1,916720,Dysphagia,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN
2,916741,Chills,23.1,Complex regional pain syndrome,23.1,Fatigue,23.1,Headache,23.1,Joint range of motion decreased,23.1
3,916741,Myalgia,23.1,Pain in extremity,23.1,Peripheral swelling,23.1,X-ray abnormal,23.1,NaN,NaN
4,916772,Feeling abnormal,23.1,SARS-CoV-2 test positive,23.1,NaN,NaN,NaN,NaN,NaN,NaN


In [418]:
VAERS_DATA_df.columns

Index(['RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX', 'DIED',
       'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL', 'HOSPDAYS', 'X_STAY',
       'DISABLE', 'RECOVD', 'VAX_DATE', 'ONSET_DATE', 'NUMDAYS', 'LAB_DATA',
       'V_ADMINBY', 'V_FUNDBY', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY',
       'PRIOR_VAX', 'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT',
       'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES'],
      dtype='object')

In [419]:
VAERS_DATA_df=VAERS_DATA_df.drop(['RECVDATE', 'STATE', 'CAGE_YR', 'CAGE_MO', 'LAB_DATA', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPDAYS', 'X_STAY', 'VAX_DATE', 'V_ADMINBY', 'V_FUNDBY', 'ER_ED_VISIT', 'OFC_VISIT', 'FORM_VERS', 'BIRTH_DEFECT', 'DISABLE', 'RECOVD', 'PRIOR_VAX', 'ONSET_DATE', 'NUMDAYS', 'SPLTTYPE', 'TODAYS_DATE'], axis=1).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,NaN,Y,Synthroid,NaN,Hypothyroidism,NKDA
916720,23.0,M,NaN,NaN,NaN,NaN,NaN,NaN
916741,68.0,F,NaN,NaN,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,NaN,NaN,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,NaN,NaN,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines","Sulfa, shellfish"


In [420]:
VAERS_DATA_df.columns

Index(['AGE_YRS', 'SEX', 'DIED', 'HOSPITAL', 'OTHER_MEDS', 'CUR_ILL',
       'HISTORY', 'ALLERGIES'],
      dtype='object')

In [421]:
VAERS_DATA_df.dtypes

AGE_YRS       float64
SEX            object
DIED           object
HOSPITAL       object
OTHER_MEDS     object
CUR_ILL        object
HISTORY        object
ALLERGIES      object
dtype: object

In [422]:
#Transform String Columns
def change_string(died):
    if died == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_df["DIED"]= VAERS_DATA_df["DIED"].apply(change_string)
VAERS_DATA_df.head(20)

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.00,F,0,Y,Synthroid,NaN,Hypothyroidism,NKDA
916720,23.00,M,0,NaN,NaN,NaN,NaN,NaN
916741,68.00,F,0,NaN,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.00,M,0,NaN,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.00,F,0,NaN,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines","Sulfa, shellfish"
916803,78.00,M,1,Y,"Lisinopril, Novolog,, Lantus Solostar, Gabapen...",none,"large T-cell lymphoma, HTN, Gout, recieving t...",N?A
916809,40.00,F,0,Y,NaN,None,NaN,None
916836,55.00,M,0,Y,No current medications,Pt reported frequent urination for several months,None,No known allergies
916859,37.00,F,0,NaN,"fexofenadine, famotidine, hydroxychloroquine,",None,"Undifferentiated connective tissue disorder, a...","peanuts, soy, oysters, crab/lobster, penicillin"


In [423]:
#Transform String Columns
def change_string1(hospital):
    if hospital == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_df["HOSPITAL"]= VAERS_DATA_df["HOSPITAL"].apply(change_string1)
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,Synthroid,NaN,Hypothyroidism,NKDA
916720,23.0,M,0,0,NaN,NaN,NaN,NaN
916741,68.0,F,0,0,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,0,0,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,0,0,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines","Sulfa, shellfish"


In [424]:
#Transform String Columns
VAERS_DATA_df['OTHER_MEDS'] = VAERS_DATA_df['OTHER_MEDS'].notnull().copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,True,NaN,Hypothyroidism,NKDA
916720,23.0,M,0,0,False,NaN,NaN,NaN
916741,68.0,F,0,0,True,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,0,0,True,NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,0,0,True,NaN,"Asthma, migraines","Sulfa, shellfish"


In [425]:
VAERS_DATA_df['OTHER_MEDS']=VAERS_DATA_df['OTHER_MEDS'].map({False: 0, True: 1}).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,1,NaN,Hypothyroidism,NKDA
916720,23.0,M,0,0,0,NaN,NaN,NaN
916741,68.0,F,0,0,1,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,0,0,1,NaN,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,0,0,1,NaN,"Asthma, migraines","Sulfa, shellfish"


In [426]:
#Transform String Columns
VAERS_DATA_df['CUR_ILL'] = VAERS_DATA_df['CUR_ILL'].notnull().copy()
VAERS_DATA_df['CUR_ILL']=VAERS_DATA_df['CUR_ILL'].map({False: 0, True: 1}).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,1,0,Hypothyroidism,NKDA
916720,23.0,M,0,0,0,0,NaN,NaN
916741,68.0,F,0,0,1,1,Rheumatoid arthritis - mostly affecting R wris...,bee stings
916772,55.0,M,0,0,1,0,"HTN, Insomnia,High Cholesterol,",Codeine
916790,52.0,F,0,0,1,0,"Asthma, migraines","Sulfa, shellfish"


In [427]:
#Transform String Columns
VAERS_DATA_df['HISTORY'] = VAERS_DATA_df['HISTORY'].notnull().copy()
VAERS_DATA_df['HISTORY']=VAERS_DATA_df['HISTORY'].map({False: 0, True: 1}).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,1,0,1,NKDA
916720,23.0,M,0,0,0,0,0,NaN
916741,68.0,F,0,0,1,1,1,bee stings
916772,55.0,M,0,0,1,0,1,Codeine
916790,52.0,F,0,0,1,0,1,"Sulfa, shellfish"


In [428]:
#Transform String Columns
VAERS_DATA_df['ALLERGIES'] = VAERS_DATA_df['ALLERGIES'].notnull().copy()
VAERS_DATA_df['ALLERGIES']=VAERS_DATA_df['ALLERGIES'].map({False: 0, True: 1}).copy()
VAERS_DATA_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
916710,23.0,F,0,1,1,0,1,1
916720,23.0,M,0,0,0,0,0,0
916741,68.0,F,0,0,1,1,1,1
916772,55.0,M,0,0,1,0,1,1
916790,52.0,F,0,0,1,0,1,1


In [429]:
##Read in 2020 data to clean and merge with 2021 data

In [430]:
file_path_2020data = "Resources/Data/2020VAERSDATA.csv"
encoding = 'unicode_escape'
VAERS_DATA_20_df = pd.read_csv(file_path_2020data, encoding='unicode_escape')
VAERS_DATA_20_df = VAERS_DATA_20_df.set_index(["VAERS_ID"]).copy()
VAERS_DATA_20_df.head()

,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
VAERS_ID,,,,,,,,,,,,,,,,,,,,,
855017,01/01/2020,HI,55.0,55.0,NaN,F,NaN,Symptoms occurred almost immediately- aching j...,NaN,NaN,...,None,Migraines,NaN,NaN,2,01/01/2020,NaN,NaN,NaN,Morphine
855018,01/01/2020,WI,68.0,68.0,NaN,F,NaN,"Extreme pain, muscle weakness in right arm so ...",NaN,NaN,...,None,"Hypothyroidism, Meniere's Disease.",NaN,NaN,2,01/01/2020,NaN,Y,Y,"Penicillin, sulfa drugs, erythromycin, tetracy..."
855019,01/01/2020,NaN,50.0,50.0,NaN,F,NaN,SORENESS IN THE AREA. ITCHING AND RASH,NaN,NaN,...,NaN,NaN,NaN,NaN,2,01/01/2020,NaN,NaN,NaN,NaN
855020,01/01/2020,TX,67.0,67.0,NaN,F,NaN,"sore arm, fever 101 , chills, and aching musc...",NaN,NaN,...,none,none,NaN,NaN,2,01/01/2020,NaN,NaN,NaN,"morphine,"
855021,01/01/2020,NaN,73.0,73.0,NaN,F,NaN,Patient presented to Clinic today 1/1/2020. Sh...,NaN,NaN,...,NaN,NaN,NaN,NaN,2,01/01/2020,NaN,Y,NaN,NaN


In [431]:
file_path_2020symptoms = "Resources/Data/2020VAERSSYMPTOMS.csv"
encoding = 'unicode_escape'
VAERS_SYMPTOMS_20_df = pd.read_csv(file_path_2020symptoms, encoding='unicode_escape')
VAERS_SYMPTOMS_20_df.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,855017,Arthralgia,22.1,Chills,22.1,Injection site pain,22.1,Pyrexia,22.1,NaN,NaN
1,855018,Chills,22.1,Fatigue,22.1,Hypertension,22.1,Hypoaesthesia,22.1,Injected limb mobility decreased,22.1
2,855018,Muscular weakness,22.1,Pain in extremity,22.1,Pyrexia,22.1,Tremor,22.1,Vertigo,22.1
3,855019,Pain,22.1,Pruritus,22.1,Rash,22.1,NaN,NaN,NaN,NaN
4,855020,Chills,22.1,Influenza like illness,22.1,Myalgia,22.1,Pain in extremity,22.1,Pyrexia,22.1


In [432]:
file_path_2020vax = "Resources/Data/2020VAERSvax.csv"
encoding = 'unicode_escape'
VAERS_VAX_20_df = pd.read_csv(file_path_2020vax, encoding='unicode_escape')
VAERS_VAX_20_df= VAERS_VAX_20_df.set_index(["VAERS_ID"]).copy()
VAERS_VAX_20_df.head()

,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
VAERS_ID,,,,,,,
855017,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,2,SYR,LA,ZOSTER (SHINGRIX)
855018,UNK,UNKNOWN MANUFACTURER,NaN,1,IM,RA,VACCINE NOT SPECIFIED (OTHER)
855019,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,NaN,NaN,ZOSTER (SHINGRIX)
855020,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,IM,LA,ZOSTER (SHINGRIX)
855021,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,2,IM,RA,ZOSTER (SHINGRIX)


In [433]:
#Begin to clean 2020 dataframes to merge later with 2021
VAERS_DATA_20_df=VAERS_DATA_20_df.drop(['RECVDATE', 'RPT_DATE', 'SYMPTOM_TEXT', 'STATE', 'CAGE_YR', 'CAGE_MO', 'LAB_DATA', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPDAYS', 'X_STAY', 'VAX_DATE', 'V_ADMINBY', 'V_FUNDBY', 'ER_ED_VISIT', 'OFC_VISIT', 'FORM_VERS', 'BIRTH_DEFECT', 'DISABLE', 'RECOVD', 'PRIOR_VAX', 'ONSET_DATE', 'NUMDAYS', 'SPLTTYPE', 'TODAYS_DATE'], axis=1).copy()
VAERS_DATA_20_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,F,NaN,NaN,Loristan,None,Migraines,Morphine
855018,68.0,F,NaN,NaN,"levothyroxine, multi vitamin, calcium.",None,"Hypothyroidism, Meniere's Disease.","Penicillin, sulfa drugs, erythromycin, tetracy..."
855019,50.0,F,NaN,NaN,NaN,NaN,NaN,NaN
855020,67.0,F,NaN,NaN,"multi vitamin, calcium, premarin cream",none,none,"morphine,"
855021,73.0,F,NaN,NaN,NaN,NaN,NaN,NaN


In [434]:
#Transform String Columns
def transform_string(died):
    if died == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_20_df["DIED"]= VAERS_DATA_20_df["DIED"].apply(transform_string)
VAERS_DATA_20_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,F,0,NaN,Loristan,None,Migraines,Morphine
855018,68.0,F,0,NaN,"levothyroxine, multi vitamin, calcium.",None,"Hypothyroidism, Meniere's Disease.","Penicillin, sulfa drugs, erythromycin, tetracy..."
855019,50.0,F,0,NaN,NaN,NaN,NaN,NaN
855020,67.0,F,0,NaN,"multi vitamin, calcium, premarin cream",none,none,"morphine,"
855021,73.0,F,0,NaN,NaN,NaN,NaN,NaN


In [435]:
#Transform String Columns
def transform_string1(hospital):
    if hospital == "Y":
        return 1
    else:
        return 0 

VAERS_DATA_20_df["HOSPITAL"]= VAERS_DATA_20_df["HOSPITAL"].apply(transform_string1)


In [436]:
#Transform String Columns

VAERS_DATA_20_df['OTHER_MEDS'] = VAERS_DATA_20_df['OTHER_MEDS'].notnull().copy()
VAERS_DATA_20_df['OTHER_MEDS']=VAERS_DATA_20_df['OTHER_MEDS'].map({False: 0, True: 1}).copy()

VAERS_DATA_20_df['CUR_ILL'] = VAERS_DATA_20_df['CUR_ILL'].notnull().copy()
VAERS_DATA_20_df['CUR_ILL']=VAERS_DATA_20_df['CUR_ILL'].map({False: 0, True: 1}).copy()

VAERS_DATA_20_df['HISTORY'] = VAERS_DATA_20_df['HISTORY'].notnull().copy()
VAERS_DATA_20_df['HISTORY']=VAERS_DATA_20_df['HISTORY'].map({False: 0, True: 1}).copy()

VAERS_DATA_20_df['ALLERGIES'] = VAERS_DATA_20_df['ALLERGIES'].notnull().copy()
VAERS_DATA_20_df['ALLERGIES']=VAERS_DATA_20_df['ALLERGIES'].map({False: 0, True: 1}).copy()

VAERS_DATA_20_df.head()


,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,F,0,0,1,1,1,1
855018,68.0,F,0,0,1,1,1,1
855019,50.0,F,0,0,0,0,0,0
855020,67.0,F,0,0,1,1,1,1
855021,73.0,F,0,0,0,0,0,0


In [437]:
#Drop Unnecessary columns from VAERS_VAX_20_df
VAERS_VAX_20_df= VAERS_VAX_20_df.drop(["VAX_TYPE", "VAX_LOT", "VAX_ROUTE", "VAX_SITE", "VAX_NAME"], axis=1).copy()

VAERS_VAX_20_df.head()

,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,
855017,GLAXOSMITHKLINE BIOLOGICALS,2
855018,UNKNOWN MANUFACTURER,1
855019,GLAXOSMITHKLINE BIOLOGICALS,UNK
855020,GLAXOSMITHKLINE BIOLOGICALS,UNK
855021,GLAXOSMITHKLINE BIOLOGICALS,2


In [438]:
len(VAERS_VAX_df.index)

5279

In [439]:
len(VAERS_DATA_df.index)

5351

In [440]:
VAERS_DATA_VAX_df= pd.merge(VAERS_DATA_df, VAERS_VAX_df, on = "VAERS_ID")
VAERS_DATA_VAX_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,,,,,,,,,
916710,23.0,F,0,1,1,0,1,1,MODERNA,1
916720,23.0,M,0,0,0,0,0,0,MODERNA,1
916741,68.0,F,0,0,1,1,1,1,PFIZER\BIONTECH,1
916772,55.0,M,0,0,1,0,1,1,PFIZER\BIONTECH,UNK
916790,52.0,F,0,0,1,0,1,1,PFIZER\BIONTECH,1


In [441]:
VAERS_DATA_VAX_20_df = pd.merge(VAERS_DATA_20_df, VAERS_VAX_20_df, on="VAERS_ID")
VAERS_DATA_VAX_20_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,,,,,,,,,
855017,55.0,F,0,0,1,1,1,1,GLAXOSMITHKLINE BIOLOGICALS,2
855018,68.0,F,0,0,1,1,1,1,UNKNOWN MANUFACTURER,1
855019,50.0,F,0,0,0,0,0,0,GLAXOSMITHKLINE BIOLOGICALS,UNK
855020,67.0,F,0,0,1,1,1,1,GLAXOSMITHKLINE BIOLOGICALS,UNK
855021,73.0,F,0,0,0,0,0,0,GLAXOSMITHKLINE BIOLOGICALS,2


In [442]:
##2020 AND 2021 VAERS DATA and VAX dataframes combined
VAERS_DATA_VAX_all_df = VAERS_DATA_VAX_20_df.append(VAERS_DATA_VAX_df)
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_MANU,VAX_DOSE_SERIES
VAERS_ID,,,,,,,,,,
855017,55.0,F,0,0,1,1,1,1,GLAXOSMITHKLINE BIOLOGICALS,2
855018,68.0,F,0,0,1,1,1,1,UNKNOWN MANUFACTURER,1
855019,50.0,F,0,0,0,0,0,0,GLAXOSMITHKLINE BIOLOGICALS,UNK
855020,67.0,F,0,0,1,1,1,1,GLAXOSMITHKLINE BIOLOGICALS,UNK
855021,73.0,F,0,0,0,0,0,0,GLAXOSMITHKLINE BIOLOGICALS,2


In [443]:
# Drop VAX_MANU Column
VAERS_DATA_VAX_all_df = VAERS_DATA_VAX_all_df.drop(["VAX_MANU"], axis=1).copy()
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,VAX_DOSE_SERIES
VAERS_ID,,,,,,,,,
855017,55.0,F,0,0,1,1,1,1,2
855018,68.0,F,0,0,1,1,1,1,1
855019,50.0,F,0,0,0,0,0,0,UNK
855020,67.0,F,0,0,1,1,1,1,UNK
855021,73.0,F,0,0,0,0,0,0,2


In [446]:
VAERS_DATA_VAX_all_df.dtypes

AGE_YRS            float64
SEX                 object
DIED                 int64
HOSPITAL             int64
OTHER_MEDS           int64
CUR_ILL              int64
HISTORY              int64
ALLERGIES            int64
VAX_DOSE_SERIES     object
dtype: object

In [447]:
# Dropping VAX_DOSE_SERIES for now, will add back later
VAERS_DATA_VAX_all_df = VAERS_DATA_VAX_all_df.drop(["VAX_DOSE_SERIES"], axis=1).copy()
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,F,0,0,1,1,1,1
855018,68.0,F,0,0,1,1,1,1
855019,50.0,F,0,0,0,0,0,0
855020,67.0,F,0,0,1,1,1,1
855021,73.0,F,0,0,0,0,0,0


In [450]:
#Transform String Column "SEX"
def no_string(gender):
    if gender == "M":
        return 1
    else:
        return 0 

VAERS_DATA_VAX_all_df["SEX"]= VAERS_DATA_VAX_all_df["SEX"].apply(no_string)
VAERS_DATA_VAX_all_df.head()

,AGE_YRS,SEX,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,,
855017,55.0,0,0,0,1,1,1,1
855018,68.0,0,0,0,1,1,1,1
855019,50.0,0,0,0,0,0,0,0
855020,67.0,0,0,0,1,1,1,1
855021,73.0,0,0,0,0,0,0,0


In [455]:
pd.get_dummies(VAERS_DATA_VAX_all_df, prefix=["GENDER"], columns= ["SEX"])

,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
VAERS_ID,,,,,,,,,
855017,55.0,0,0,1,1,1,1,1,0
855018,68.0,0,0,1,1,1,1,1,0
855019,50.0,0,0,0,0,0,0,1,0
855020,67.0,0,0,1,1,1,1,1,0
855021,73.0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
1024788,90.0,1,0,0,0,0,0,1,0
1024790,91.0,1,0,0,0,0,0,1,0
1024795,86.0,1,0,0,0,0,0,1,0


In [462]:
### Logistic Regression Model to predict death
y = VAERS_DATA_VAX_all_df["DIED"]
X = VAERS_DATA_VAX_all_df.drop(["DIED"], axis=1)
X.head()

,AGE_YRS,SEX,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES
VAERS_ID,,,,,,,
855017,55.0,0,0,1,1,1,1
855018,68.0,0,0,1,1,1,1
855019,50.0,0,0,0,0,0,0
855020,67.0,0,0,1,1,1,1
855021,73.0,0,0,0,0,0,0


In [466]:
X.dtypes

AGE_YRS       float64
SEX             int64
HOSPITAL        int64
OTHER_MEDS      int64
CUR_ILL         int64
HISTORY         int64
ALLERGIES       int64
dtype: object

In [468]:
VAERS_DATA_VAX_all_df.AGE_YRS=VAERS_DATA_VAX_all_df.AGE_YRS.fillna(0)

In [469]:
VAERS_DATA_VAX_all_df.AGE_YRS=VAERS_DATA_VAX_all_df.AGE_YRS.astype(int)

In [474]:
VAERS_DATA_VAX_all_df.dtypes

AGE_YRS       int32
SEX           int64
DIED          int64
HOSPITAL      int64
OTHER_MEDS    int64
CUR_ILL       int64
HISTORY       int64
ALLERGIES     int64
dtype: object

In [470]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [471]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [477]:
classifier.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [473]:
y_pred = classifier.predict(X_test)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [109]:
###############

In [110]:
##Initializing model with K=3
model = KMeans(n_clusters=3, random_state=5)
model

KMeans(n_clusters=3, random_state=5)

In [ ]:
# Fitting model
model.fit(VAERS_DATA_df)

In [ ]:
# Get the predictions
predictions = model.predict(VAERS_DATA_df)
print(predictions)

In [ ]:
# Add a new class column to the VAERS_DATA_df
VAERS_DATA_df["class"] = model.labels_
VAERS_DATA_df.head()

In [ ]:
# Creating a scatterplot of VAERS_DATA_df
#Using features 
df_iris.hvplot.scatter(x="SEX", y="HOSPITAL", by="class")

In [ ]:
#Plotting the clusters with three features
fig = px.scatter_3d(
VAERS_DATA_df,
x="SEX",
y="HOSPITAL",
z="CUR_ILL",
color="class",
symbol="class",
size="sepal_width",
width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
#Function to cluster and plot dataset
def test_cluster_amount(df, clusters):
    model = KMeans(n_clusters=clusters, random_state=5)
    model

    # Fitting model
    model.fit(df)

    # Add a new class column to VAERS_DATA_DF
    df["class"] = model.labels_

In [ ]:
##Test Cluster Amounts
test_cluster_amount(VAERS_DATA_df, 2)
VAERS_DATA_df.hvplot.scatter(x="SEX", y="HOSPITAL", by="class")

In [ ]:
## Use Elbow Curve to find best number of clusters
# Create empty list to hold inertia values
# Store a range of K values to test

inertia = []
k = list(range(1,11))

In [ ]:
# Looking for the best K
for i in k:
km = KMeans(n_clusters=i, random_state=0)
    km.fit(VAERS_DATA_df)
inertia.append(km.inertia_)

In [ ]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

In [ ]:
##Look at the graph generated and determine correct amount of clusters

In [ ]:
# Create a K-means function to reuse the K-means cluster
def get_clusters(k, data):
   # Create a copy of the DataFrame
   data = data.copy()

   # Initialize the K-Means model
   model = KMeans(n_clusters=k, random_state=0)

   # Fit the model
   model.fit(data)

   # Predict clusters
   predictions = model.predict(data)

   # Create return DataFrame with predicted clusters
   data["class"] = model.labels_

   return data